# Titanic dataset

Pierce Renio
Purpose is to test and hone my data science skills

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OrdinalEncoder # for encoding categorical features from strings to number arrays
from sklearn.naive_bayes import MultinomialNB, CategoricalNB

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [50]:
df = pd.read_csv("train.csv")
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [51]:
df = df.drop(columns = df.columns[0])
df = df.drop(columns = df.columns[2])
df = df.drop(columns = df.columns[6])
df = df.drop(columns = df.columns[7])
df["Sex"] = df["Sex"].replace(["female","male"],[0,1])
df["Embarked"] = df["Embarked"].replace(['S','C','Q'],[1,2,3])
df = df.dropna()

print(df.shape)
df.head()

(712, 8)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,1.0
1,1,1,0,38.0,1,0,71.2833,2.0
2,1,3,0,26.0,0,0,7.9250,1.0
3,1,1,0,35.0,1,0,53.1000,1.0
4,0,3,1,35.0,0,0,8.0500,1.0


In [52]:
df['Survived'].value_counts(normalize=True)

0    0.595506
1    0.404494
Name: Survived, dtype: float64

In [53]:
#Setting the seed
np.random.seed(1234)

#Randomizing dataset
data_randomized = df.sample(frac=1)

#Calcculating index for split (80/20)
trainsize = round(len(data_randomized) * 0.8)

#Splitting the data
training_set = data_randomized[:trainsize].reset_index(drop=True)
test_set = data_randomized[trainsize:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(570, 8)
(142, 8)


Checking to see if the testing and training data sets are about the same proportion


In [54]:
training_set["Survived"].value_counts(normalize=True)

0    0.6
1    0.4
Name: Survived, dtype: float64

In [55]:
test_set["Survived"].value_counts(normalize=True)

0    0.577465
1    0.422535
Name: Survived, dtype: float64

The proportions are about the same. Sanity check completed.

## Creating the Naive Bayes model


In [56]:
trainX = training_set.iloc[:,1:]
trainy = training_set["Survived"]

colnames = trainX.columns

trainX.head()
trainy.head()

0    1
1    0
2    0
3    0
4    1
Name: Survived, dtype: int64

In [57]:
test_set.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,1,0,38.0,0,0,227.5250,2.0
1,1,3,1,12.0,1,0,11.2417,2.0
2,0,2,0,38.0,0,0,13.0000,1.0
3,1,1,1,48.0,1,0,76.7292,2.0
4,0,3,1,27.0,0,0,7.8958,1.0


In [58]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #rename function for simpler writing

#0-1 encoding train labels (think of it as a Bernoulli variable)
trainBrnli = le.fit_transform(trainy)

trainBrnli[:5] #print first 5 of train Bernoulli (check that No=0, Yes=1)

array([1, 0, 0, 0, 1])

### Creating and fitting the model

In [59]:
model = CategoricalNB()
model.fit(trainX,trainBrnli)

CategoricalNB()

In [60]:
yhattrain = model.predict(trainX)

In [61]:
pd.crosstab(yhattrain,trainy)

Survived,0,1
row_0,,
0,306,58
1,36,170


In [62]:
accuracy_score(yhattrain,trainBrnli)

0.8350877192982457

### For the test data

In [64]:
testX = test_set.iloc[:,1:]
testy = test_set["Survived"]

testX.head()
testy.head()

0    1
1    1
2    0
3    1
4    0
Name: Survived, dtype: int64

In [68]:
testBrnli = le.fit_transform(testy)

testBrnli[:5]


yhattest = model.predict(testX)


In [69]:
confusion_matrix(yhattest, testBrnli)

array([[52, 14],
       [30, 46]])

In [70]:
accuracy_score(yhattest, testBrnli)

0.6901408450704225